In [1]:
import talib
from data.Symbols import healthy_shares

symbols = healthy_shares()
interval = 1440

patterns = talib.get_function_groups()['Pattern Recognition']

In [8]:
from datetime import datetime
from data.TimescaleDBSticksDao import get_sticks

def score_of_symbol(symbol: str, from_date: datetime, to_date: datetime):
    sticks = get_sticks(symbol, interval, from_date, to_time=to_date)

    scores_of_symbol = {}
    for pattern in patterns:
        # below is same as;
        pattern_scores = getattr(talib, pattern)(sticks['ask_open'], sticks['ask_high'], sticks['ask_low'], sticks['ask_close'])
        if pattern_scores.iloc[-1] != 0:
            if scores_of_symbol:
                scores_of_symbol[pattern] = pattern_scores.iloc[-1]
            else:
                scores_of_symbol = {pattern: pattern_scores.iloc[-1]}
    return scores_of_symbol

In [8]:
from data.TimescaleDBSticksDao import get_sticks
from datetime import timedelta
from pandas import Timestamp

scores = {}
for symbol in symbols:
    try:
        scores[symbol] = score_of_symbol(symbol, datetime.today())
    except Exception:
        print(f"something wrong with ${symbol}")
        continue

sorted_scores = sorted(scores.items(), key=lambda x:sum(x[1].values()), reverse=True)

AttributeError: 'NoneType' object has no attribute 'values'

In [13]:
from data.Symbols import load_symbols_info

dct = load_symbols_info()
sushi_symbol = [key for key, value in dct.items() if 'Sushiro' in value][0]
print(sushi_symbol)
sushi_score = score_of_symbol(sushi_symbol,datetime.strptime('2020-04-27 14:30:00', '%Y-%m-%d %H:%M:%S'), datetime.strptime('2023-04-10 14:30:00', '%Y-%m-%d %H:%M:%S'))
sushi_score

AJ.D.3563JT.DAILY.IP


{'CDLSPINNINGTOP': 100}

In [4]:
print(sushi_score)

None


In [25]:
import pandas as pd
from data.Symbols import basePath
import json

with open(f"{basePath}/dfb-share-list", 'r') as file:
    symbol_infos = pd.DataFrame(json.load(file))

# Top trending up candlestick pattern
up_signs = sorted_scores[0:5]
[print(symbol_infos[symbol_infos.epic == up_sign[0]].instrumentName)  for up_sign in up_signs]

7606    Frontier Communications Parent Inc
Name: instrumentName, dtype: object
10139    Sixth Street Specialty Lending
Name: instrumentName, dtype: object
10639    Toro Corp
Name: instrumentName, dtype: object
12815    OceanaGold Corporation
Name: instrumentName, dtype: object
13234    Schneider Electric SA
Name: instrumentName, dtype: object


[None, None, None, None, None]

In [34]:
# Top trending down candlestick pattern
down_signs = sorted_scores[-6:-1]
[print(symbol_infos[symbol_infos.epic == s[0]].instrumentName)  for s in down_signs]

10889    Unum Group
Name: instrumentName, dtype: object
15558    Otsuka Holdings Co Ltd
Name: instrumentName, dtype: object
15862    Tokyo Broadcasting System Holdings Inc
Name: instrumentName, dtype: object
10718    TRI Pointe Group Inc
Name: instrumentName, dtype: object
14091    GF Securities Co Ltd
Name: instrumentName, dtype: object


[None, None, None, None, None]

In [8]:
from swing_trainer.main import plot_chart

symbol = sorted_scores[1][0]
sticks = get_sticks(symbol, interval)
plot_chart(symbol, sticks.iloc[:100])

IndexError: list index out of range